In [114]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

import tqdm
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
from tinydb import TinyDB, Query
from operator import itemgetter
from utils import read_config
import googleapiclient.discovery

import warnings
warnings.filterwarnings("ignore")

config = read_config('config.json')
start_transaction = lambda: TinyDB('data/not_trending_db.json')

In [115]:
df = pd.read_pickle('data/chkp1.pkl')

with start_transaction() as db:
    video_ids = set(map(itemgetter('video_id'), db.all()))
    print("Liczba rekordów:", len(video_ids))

    inserted = db.insert_multiple({'video_id': idx} for idx in df.index if idx not in video_ids)
    print("Wstawiono:", len(inserted))

Liczba rekordów: 0
Wstawiono: 8606


In [116]:
minDate = df.publish_time.min().isoformat()
maxDate = df.publish_time.max().isoformat()
print("Wyszukiwanie danych od:", minDate, "do", maxDate)

Wyszukiwanie danych od: 2006-07-23T08:24:11+00:00 do 2018-06-14T01:31:53+00:00


In [113]:
class YTDownloader:
    def __init__(self, apikey):
        self.youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=apikey)

    def get_id(self, title, channel):
        request = self.youtube.search().list(
            part="snippet",
            maxResults=1,
            q=f"{title} {channel}")
        response = request.execute()
        print(f'For {title} and {channel} the response is: {response}')
        return response

    def details_by_id(self, idx):
        request = self.youtube.videos().list(
            part="snippet",
            id=','.join(idx)
        )
        return request.execute()
    
    def find_related(self, idx, minDate, maxDate):
        request = self.youtube.search().list(
            part="snippet",
            maxResults=100,
            publishedAfter="2006-07-23T08:24:11+00:00",
            publishedBefore="2018-06-14T01:31:53+00:00",
            relatedToVideoId=idx,
            type="video"
        )
        return request.execute()

In [112]:
Video = Query()

with start_transaction() as db:
    doc = db.get(~(Video.related.exists()))
    print(doc['video_id'])
    doc['related'] = 'related'
    db.update(doc, Video.video_id == doc['video_id'])

wnwF1FHybDQ


#### Wczytywanie danych

In [122]:
with start_transaction() as db:
    doc = db.search(Video.related.exists())
    print(doc)

[{'video_id': '9wRQljFNDW8', 'related': {'kind': 'youtube#searchListResponse', 'etag': '5388FcXxgweoWorJU6IdhEsTS6E', 'nextPageToken': 'CDIQAA', 'regionCode': 'PL', 'pageInfo': {'totalResults': 102, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'YFO8wLWNjKE8sveySSuUHGLqISg', 'id': {'kind': 'youtube#video', 'videoId': '91GGoIvudBQ'}, 'snippet': {'publishedAt': '2018-01-12T11:21:46Z', 'channelId': 'UCXot-Ed4sV20SxUbC0mEnuw', 'title': 'Dion Lewis 2017 NFL Season Highlights', 'description': 'Stats:\n\nGames: 16\nTotal Yards: 1,680\nTotal Touchdowns: 10\n\nRushing: \n\nAttempts: 180\nYards: 896\nAverage Yards Per Game: 56.0\nAverage Yards Per Carry: 5.0\nTouchdowns: 6\n\nReceiving: \n\nReceptions: 32\nYards: 214\nAverage Yards Per Game: 13.4\nAverage Yards Per Reception: 6.7\nTouchdowns: 3\n\nKick Return:\n\nReturns: 23\nAverage Yards Per Return: 24.8\nYards: 570\nLongest Return: 103 Yards\nTouchdowns: 1', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi